# Sentiment Classification

Material by Andrew Trask

- **Twitter**: @iamtrask
- **Blog**: http://iamtrask.github.io

Edits & solution by Andrei Dyomin

 - **GitHub**: https://github.com/adyomin

### What You Should Already Know

- neural networks, forward and back-propagation
- stochastic gradient descent
- mean squared error
- and train/test splits

### Tutorial Outline:

- Intro: The Importance of "Framing a Problem"


- Curate a Dataset
- Developing a "Predictive Theory"
- **PROJECT 1**: Quick Theory Validation


- Transforming Text to Numbers
- **PROJECT 2**: Creating the Input/Output Data


- Putting it all together in a Neural Network
- **PROJECT 3**: Building our Neural Network


- Understanding Neural Noise
- **PROJECT 4**: Making Learning Faster by Reducing Noise


- Analyzing Inefficiencies in our Network
- **PROJECT 5**: Making our Network Train and Run Faster


- Further Noise Reduction
- **PROJECT 6**: Reducing Noise by Strategically Reducing the Vocabulary


- Analysis: What's going on in the weights?

# Lesson: Curate a Dataset

In [34]:
def pretty_print_review_and_label(i):
    print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

f = open('./data/reviews.txt','r') # What we know!
reviews = [line.strip('\n') for line in f]
f.close()

f = open('./data/labels.txt','r') # What we WANT to know!
labels = [line.strip('\n').upper() for line in f]
f.close()

In [35]:
len(reviews)

25000

In [36]:
reviews[0]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   '

In [37]:
labels[0]

'POSITIVE'

# Lesson: Develop a Predictive Theory

In [6]:
print("labels.txt \t : \t reviews.txt\n")
pretty_print_review_and_label(2137)
pretty_print_review_and_label(12816)
pretty_print_review_and_label(6267)
pretty_print_review_and_label(21934)
pretty_print_review_and_label(5297)
pretty_print_review_and_label(4998)

labels.txt 	 : 	 reviews.txt

NEGATIVE	:	this movie is terrible but it has some good effects .  ...
POSITIVE	:	adrian pasdar is excellent is this film . he makes a fascinating woman .  ...
NEGATIVE	:	comment this movie is impossible . is terrible  very improbable  bad interpretat...
POSITIVE	:	excellent episode movie ala pulp fiction .  days   suicides . it doesnt get more...
NEGATIVE	:	if you haven  t seen this  it  s terrible . it is pure trash . i saw this about ...
POSITIVE	:	this schiffer guy is a real genius  the movie is of excellent quality and both e...


# Project 1: Quick Theory Validation

In [7]:
from collections import Counter
import numpy as np
import re

In [8]:
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

In [9]:
# Using Python's Regular Expressions library to filter out all 'small' words

for review, label in zip(reviews, labels):
    if label == 'POSITIVE':
            positive_counts.update(re.findall('\w{3,}', review.lower()))
    else:
            negative_counts.update(re.findall('\w{3,}', review.lower()))

total_counts.update(positive_counts)
total_counts.update(negative_counts)

## Creating a stop list

Filtering out https://en.wikipedia.org/wiki/Stop_words

In [10]:
f = open('./data/stop_list.csv','r')
stop_list_general = set(f.read().split(', '))
f.close()
print('stop_list_general - {0} words'.format(len(stop_list_general)))

stop_list_reviews = set()
for word, count in total_counts.most_common():
    if count >= 2000:
        stop_list_reviews.add(word)
len(stop_list_reviews)

print('stop_list_reviews - {0} words'.format(len(stop_list_reviews)))

stop_list_general - 119 words
stop_list_reviews - 291 words


In [11]:
# super subjective list
keep_words = set('bad, dvd, nothing, old, pretty, again, beautiful, best, big, excellent, first, fun, funny, good, great, horror, interesting, little, like, love, many, money, most, never, must, nice, off, original, point, special, star, true, very, well, worst'.split(', '))
len(keep_words)

35

In [12]:
full_stop_list = stop_list_general | stop_list_reviews - keep_words
len(full_stop_list)

300

In [57]:
# f = open('./data/full_stop_list.txt', 'r+')
# for word in full_stop_list:
#     f.writelines('{0}'.format(word))
# f.close()

In [13]:
for word in full_stop_list:
    del positive_counts[word]
    del negative_counts[word]
    del total_counts[word]

In [14]:
pos_neg_ratios = Counter()
threshold = 256
max_ratio = 3
max_ratio_raw = np.exp(max_ratio)

for word, count in total_counts.most_common():
    if (count > threshold) & (negative_counts[word] > 0) :
        pos_neg_ratio = positive_counts[word]/negative_counts[word]
    elif (count > threshold) & (negative_counts[word] == 0) :
        pos_neg_ratio = max_ratio_raw
    pos_neg_ratios[word] = pos_neg_ratio
    
for word, ratio in pos_neg_ratios.most_common():
    if ratio > 0:
        pos_neg_ratios[word] = np.log(ratio)
    elif ratio == 0:
        print (word)
        pos_neg_ratios[word] = -max_ratio

In [15]:
# words most frequently seen in a review with a "POSITIVE" label
pos_neg_ratios.most_common(10)

[('wonderfully', 2.0485643031153966),
 ('delightful', 1.8262456452992242),
 ('beautifully', 1.7784436932522829),
 ('superb', 1.7189076208420597),
 ('touching', 1.6514021115331325),
 ('stewart', 1.6249021381316819),
 ('friendship', 1.588384503236268),
 ('magnificent', 1.5686159179138452),
 ('wonderful', 1.5680329974659779),
 ('finest', 1.5668782980153044)]

In [16]:
# words most frequently seen in a review with a "NEGATIVE" label
list(reversed(pos_neg_ratios.most_common()))[0:10]

[('unfunny', -2.6882475738060303),
 ('waste', -2.6186484579840514),
 ('pointless', -2.4531579514734201),
 ('redeeming', -2.3648889763302003),
 ('worst', -2.2865847516476046),
 ('laughable', -2.2617630984737906),
 ('awful', -2.2265521924307397),
 ('poorly', -2.2192034840549946),
 ('sucks', -1.9830278120118159),
 ('lame', -1.9802348915963879)]

# Transforming Text into Numbers

**review** = "This was a horrible, terrible movie."

<img src = './data/sentiment_network.png'>

**review** = "The movie was excellent"

<img src = './data/sentiment_network_pos.png'>

# Project 2: Creating the Input/Output Data

In [17]:
vocab = set(total_counts.keys())
vocab_size = len(vocab)
print(vocab_size)

73297


In [18]:
import numpy as np

layer_0 = np.zeros((1,vocab_size))
layer_0

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])

<img src = './data/sentiment_network.png'>

In [19]:
word2index = {}

for i, word in enumerate(vocab):
    word2index[word] = i

In [21]:
%%time

limit = 127

features = np.zeros((limit, vocab_size))
for i, review in enumerate(reviews[:limit]):
    for word in review.split():
        if word in vocab:
            features[i, word2index[word]] += 1
            
features.shape

CPU times: user 24.2 ms, sys: 14.5 ms, total: 38.6 ms
Wall time: 38.3 ms


In [22]:
print(features.shape)
labels = np.array(labels[:limit], ndmin=2).T
print(labels.shape)

(127, 73297)
(127, 1)


# Project 3: Building a Neural Network

- Start with your neural network from the last chapter
- 3 layer neural network
- no non-linearity in hidden layer
- use our functions to create the training data
- create a "pre_process_data" function to create vocabulary for our training data generating functions
- modify "train" to train over the entire corpus

In [23]:
import sys
nn_path = '/Users/adyomin/Yandex.Disk.localized/Projects/Cornerstone'
sys.path.append(nn_path)
import network as nn

In [24]:
help(nn.Network.__init__)

Help on function __init__ in module network:

__init__(self, size, h_activation='sigmoid', o_activation='pass_input', c_function='quadratic', weights=None)
    Network class constructor method.
    
    Parameters
    ----------
    size : tuple
        size[0] - n_features, input layers width.  size[-1] - n_targets,
        width of the output layer.
    
    h_activation : string
        Choice of activation function for all hidden layers.  Current
        options include:
         - 'sigmoid' - returns 1/(1 + numpy.exp(-x))
         - 'pass_input' - returns x
    
    o_activation : string
        Choice of output layer activation function.  Current options
        include:
         - 'sigmoid' - returns 1/(1 + numpy.exp(-x))
         - 'pass_input' - returns x
    
    c_function : string
        Choice of cost/loss/objective function prime for the network.
        Current options include:
         - 'quadratic' - returns error, (0.5*(error**2))' = error
    
    weights : numpy.ar

In [25]:
nn_model = nn.Network((73297, 256, 64, 1))

In [26]:
help(nn.Network.train)

Help on function train in module network:

train(self, x_train, y_train, batch_size, eta, n_epochs, shuffle=True)
    Trains the instance with its current weights to predict y from x.
    
    Parameters
    ----------
    x_train : numpy.array
        Train examples matrix of size (n_records, n_features).  Matrix
        elements are expected to be real numbers with mean = 0.0, scaled
        to 1.0 standard deviation.
    
    y_train : numpy.array
        Train labels matrix of size (n_records, n_targets).
    
    batch_size : int
        Defines how many train examples will be taken for the next step of
        the weights update.  Would be great to add some intuition how to
        choose a batch size.  TBD I guess.
    
    n_epochs : int
        Number of epochs to train over the whole x.
    
    eta : float
        Weight update step multiple.  Constant only ATM.
    
    shuffle: bool
        Determines whether or not data is being shuffled each epoch.



In [28]:
# nn_model.train(features[:128], labels[:128], batch_size=16, eta=0.01, n_epochs=1)